In [1]:
import json
import os
import random
import fnmatch
import openpyxl

from urllib.request import urlopen
import urllib

import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np


%matplotlib inline

In [2]:
sns.set_style("darkgrid")

In [3]:
def import_json_to_dict(url) :
    response = urllib.request.urlopen(url)
    my_dict = json.loads(response.read())
    return my_dict

In [4]:
def clean_str(s):
    d = {
        "’": "'",
        "\xa0": " "
        }
    for x in d:
        s = s.replace(x, d[x]).strip()
    return s

In [5]:
month_dict = {
    "January" : "Janvier",
"February": "Février",
"March": "Mars",
"April" : "Avril",
"May" : "Mai",
"June" : "Juin",
"July" : "Juillet",
"August" : "Août",
"September":"Septembre",
"October":"Octobre",
"November":"Novembre",
"December":"Décembre"}

In [6]:
data_dir_path = './data/'

In [7]:
#Taxonomies

taxo_dep_url = 'https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/taxonomies/departements-minify.json'
taxo_reg_url = 'https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/taxonomies/regions-minify.json'

In [8]:
mailles_list = ['national', 'regional', 'departemental']

In [9]:
propilot_path = os.path.join("data")

In [10]:
!ls $propilot_path

'CSV - Plan de Relance'   dim_properties.csv	 dim_tree_nodes.csv
 dim_activity.csv	  dim_snapshots.csv	 fact_financials_enr_pp_ch.csv
 dim_effects.csv	  dim_states_pp_ch.csv	 fact_property_values.csv
 dim_maturities.csv	  dim_structures.csv	 old
 dim_period_pp_ch.csv	  dim_top_levels.csv


In [11]:
df_dict = {}
file_list = [
"dim_activity",
"dim_tree_nodes",
"dim_top_levels",
"dim_maturities",
"dim_period",
"dim_snapshots",
"dim_effects",
"dim_properties",
"dim_states",
"dim_structures",
"fact_financials",
"fact_property_values"
]

In [13]:
for file in file_list:
    for file_csv in os.listdir(propilot_path):
        if fnmatch.fnmatch(file_csv, file + "*.csv"):
            print("File loaded : ", file_csv)
            df_dict[file] = pd.read_csv(os.path.join(propilot_path, file_csv), sep=";")

File loaded :  dim_activity.csv
File loaded :  dim_tree_nodes.csv
File loaded :  dim_top_levels.csv
File loaded :  dim_maturities.csv
File loaded :  dim_period_pp_ch.csv
File loaded :  dim_snapshots.csv
File loaded :  dim_effects.csv
File loaded :  dim_properties.csv
File loaded :  dim_states_pp_ch.csv
File loaded :  dim_structures.csv
File loaded :  fact_financials_enr_pp_ch.csv
File loaded :  fact_property_values.csv


In [14]:
df = (df_dict["fact_financials"]
      .merge(df_dict["dim_tree_nodes"], left_on="tree_node_id", right_on="tree_node_id")
      .merge(df_dict["dim_effects"], left_on="effect_id", right_on="effect_id")
      .merge(df_dict["dim_states"], left_on="state_id", right_on="state_id")
      .merge(df_dict["dim_period"], left_on="period_id", right_on="period_id")
      .merge(df_dict["dim_structures"], left_on="structure_id", right_on="structure_id"))

In [15]:
cols = ["tree_node_name", "structure_name", "effect_id", "period_date", "period_month_tri", "period_month_year", "financials_cumulated_amount"]
df=df[cols]
df.rename(columns={"period_month_year":"Date", "financials_cumulated_amount":"valeur"}, inplace=True)

In [16]:
forbidden_period_value = ["Y", "Total"]

In [17]:
df_dep = df.loc[
    (df.structure_name == "Département") & 
    (~df.period_month_tri.isin(forbidden_period_value))].copy()

In [18]:
df_dep["departement"] = df_dep["tree_node_name"].apply(lambda x: x.split("/")[0].strip())
df_dep["mesure"] = df_dep["tree_node_name"].apply(lambda x: x.split("/")[1].strip())

# nettoyage de la colonne mesure, on enlève un point surnuméraire.
df_dep["mesure"].replace("\.", "", regex=True,inplace=True)
df_dep.mesure = df_dep.mesure.apply(lambda x: clean_str(x))

df_dep.rename(columns={"effect_id":"indicateur"}, inplace=True)
df_dep.indicateur = df_dep.indicateur.str.strip()
# traduit les mois dans la colonne date
df_dep.Date = df_dep.Date.replace(month_dict, regex=True)

df_dep.valeur.replace({",": "."}, regex=True, inplace=True)
df_dep.valeur = df_dep.valeur.astype(float)

In [41]:
df_dep["short_indic"] = df_dep.indicateur.apply(lambda x: x.split("-")[0].strip())
df_dep.short_indic = df_dep.short_indic.apply(lambda x: clean_str(x))

In [59]:
dict_indicateur = {'Nombre de repas servis dans les restaurants universitaires au tarif à 1€' : "Nombre de repas servis",
 'Montant cumulé de l’investissement total ainsi déclenché' : "Montant cumumé de l'investissement total",
 'Nombre d’entreprises bénéficiaires du dispositif' : "Nombre d'entreprises bénéficiaires",
 'Nombre de bâtiments Etat dont le marché de rénovation est notifié' : 'Nombre de bâtiments dont le marché de rénovation est notifié',
 'Nombre de projets incluant une transformation de la ligne de production pour réduire son impact environnemental' : 'Nombre de projets',
 'Nombre d\'exploitations certifiées "haute valeur environnementale"' : 'Nombre d’exploitations certifiées',
 'Emissions de gaz à effet de serre évitées sur la durée de vie des équipements' : 'Emissions de gaz à effet de serre évitées',
 'Nombre de bonus octroyés à des véhicules électriques et hybrides rechargeables' : 'Nombre de bonus octroyés à des véhicules électriques',
 "Quantité de matières plastiques évitées ou dont le recyclage ou l'intégration a été soutenue" : 'Quantité de matières plastiques évitées',
 'Montant total des travaux associés aux dossiers validés' : 'Montant total des travaux',
 'Nombre de nouveaux projets (nouvelle ligne, extension de ligne et pôle)' : 'Nombre de nouveaux projets',
 'Montant de l’investissement total déclenché' : 'Montant de l’investissement total',
 'Nombre de projets de tourisme durable financés' : 'Nombre de projets de tourisme durable financés',
 'Nombre de projets de rénovation de cathédrales et de monuments nationaux initiés' : 'Nombre de projets de rénovation',
 'Montant total investi pour la rénovation de monuments historiques appartenant aux collectivités territoriales' :'Montant total investi pour la rénovation',
 'Nombre de projets de rénovation de monuments historiques appartenant aux collectivités territoriales bénéficiaires initiés' : 'Nombre de projets de rénovation',
 "Nombre de contrats de professionnalisation bénéficiaires de l'aide exceptionnelle" : 'Nombre de contrats de professionnalisation',
 "Nombre de contrats d'apprentissage bénéficiaires de l'aide exceptionnelle" : 'Nombre de contrats d’apprentissage',
 "Nombre d'aides à l'embauche des travailleurs handicapés": "Nombre d'aides à l'embauche des travailleurs handicapés",
 'Nombre de projets locaux soutenus  (rénovation, extension, création de lignes' :'Nombre de projets locaux soutenus'
                  }

In [60]:
df_dep.short_indic = df_dep.short_indic.apply(lambda x: dict_indicateur[x] if x in dict_indicateur else x)

In [61]:
[x for x in list(df_dep.short_indic.unique()) if len(x) >40]

["Montant cumulé de l'investissement total ainsi déclenché",
 "Nombre d'entreprises bénéficiaires du dispositif",
 'Nombre de bâtiments dont le marché de rénovation est notifié',
 'Emissions de gaz à effet de serre évitées',
 'Nombre de bonus octroyés à des véhicules électriques',
 "Montant de l'investissement total déclenché",
 'Nombre de projets de tourisme durable financés',
 'Nombre de contrats de professionnalisation',
 "Nombre d'aides à l'embauche des travailleurs handicapés"]

In [62]:
dict_mesures  = {
 "Appels à projets pour les projets d'efficacité énergétique de grande ampleur et appels à manifestation d'intérêt sur les projets qui proposent une évolution profonde des procédés industriels": "AAP et AMI Efficacité énergétique",
 "CIE jeunes" : "Contrats Initiatives Emploi (CIE) Jeunes",
 'France Num' : 'France Num : aide à la numérisation des TPE,PME,ETI',
 'Guichet efficacité énergétique dans industrie' : 'Guichet efficacité énergétique',
 "Modernisation de la filière (BPI)" : "AAP industrie : Modernisation des filières auto et aéro",
 "PEC jeunes": "Parcours emploi compétences (PEC) Jeunes",
 'Relocalisation : soutien aux projets industriels dans les territoires' : 'AAP Industrie : Soutien aux projets industriels territoires',
 'Relocalisation : sécurisation des approvisionnements critiques' : 'AAP Industrie : Sécurisation approvisionnements critiques',
 'Renforcement des subventions de Business France (chèque export, chèque VIE)' : 'Renforcement subventions Business France',
 "Soutien à la modernisation industrielle et renforcement des compétences dans la filière nucléaire" : "AAP industrie : modernisation industrielle et renforcement des compétences dans la filière nucléaire",
 'Soutien à la recherche aéronautique civil' :'Soutien recherche aéronautique civil',
 'Rénovation bâtiments Etats' : 'Rénovation des bâtiments Etats (marchés notifiés)'
}

In [63]:
df_dep["short_mesure"] = df_dep.mesure.apply(lambda x: dict_mesures[x] if x in dict_mesures else x)

In [64]:
taxo_dep_url = 'https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/taxonomies/departements-minify.json'
taxo_reg_url = 'https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/taxonomies/regions-minify.json'
taxo_dep_df = pd.DataFrame(import_json_to_dict(taxo_dep_url))
dep_list = list(taxo_dep_df['dep'])
print('{} departements.'.format(len(dep_list)))

taxo_reg_df = pd.DataFrame(import_json_to_dict(taxo_reg_url))
reg_list = list(taxo_reg_df['reg'])
print('{} regions.'.format(len(reg_list)))

101 departements.
18 regions.


In [65]:
df_dep_enr = df_dep.merge(taxo_dep_df[["dep", "reg", "libelle"]], 
                          how="left", left_on="departement", right_on="libelle")\
    .merge(taxo_reg_df[["reg", "nccenr"]], how="left", left_on="reg", right_on="reg")
df_dep_enr.drop(columns=["tree_node_name", "structure_name"], inplace=True)
df_dep_enr = df_dep_enr.loc[~df_dep_enr.dep.isna(), :]
df_dep_enr.rename(columns={"nccenr":"region"}, inplace=True)
df_dep_enr.shape

(13395, 13)

In [66]:
df_dep_enr.to_csv("pp_dep.csv", sep=";")